<a href="https://colab.research.google.com/github/harshith7823/CS760-Project/blob/reetu/part-1/Death%20prediction/Model%20generated%20death%20column/part1_model_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from numpy import mean
from numpy import std
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, ElasticNetCV,RidgeCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

In [3]:
oppScrData = pd.read_excel(r'sample_data/OppScrData.xlsx')

In [4]:
def clean_ct_data(oppScrData):
    # Delete rows with empty values
    cols = ["L1_HU_BMD", "TAT Area (cm2)", 'Total Body                Area EA (cm2)',
       'VAT Area (cm2)', 'SAT Area (cm2)', 'VAT/SAT     Ratio', 'Muscle HU',
       ' Muscle Area (cm2)', 'L3 SMI (cm2/m2)', 'AoCa        Agatston',
       'Liver HU    (Median)', 'DEATH [d from CT]']
    ct_data= oppScrData[cols]
    n = ct_data.shape[0]
    preprocessed_ct_data = pd.DataFrame( columns=cols)
    for i in range(n):
        row = ct_data.loc[i]
        ignore = False
        for k,j in enumerate(row):
          if pd.isna(j) or j == ' ': 
            if k == 11:              
              ignore = False
            else:
              ignore = True
              break
        if not ignore:
          preprocessed_ct_data.loc[i]= row
    return preprocessed_ct_data

In [ ]:
sc = StandardScaler()
ct_data = clean_ct_data(oppScrData=oppScrData)

In [13]:
non_null_death_rows = ct_data[~ct_data['DEATH [d from CT]'].isnull()]
print(non_null_death_rows.shape)
print(non_null_death_rows.columns)

(493, 12)
Index(['L1_HU_BMD', 'TAT Area (cm2)',
       'Total Body                Area EA (cm2)', 'VAT Area (cm2)',
       'SAT Area (cm2)', 'VAT/SAT     Ratio', 'Muscle HU',
       ' Muscle Area (cm2)', 'L3 SMI (cm2/m2)', 'AoCa        Agatston',
       'Liver HU    (Median)', 'DEATH [d from CT]'],
      dtype='object')


In [ ]:
# Convert to years
non_null_death_rows['DEATH [d from CT]'] /= 365

X = non_null_death_rows.iloc[:, 0:11]
y = non_null_death_rows.iloc[:, 11]
X = pd.DataFrame(sc.fit_transform(X), columns = X.columns)

null_death_rows=ct_data[ct_data['DEATH [d from CT]'].isnull()]
X2 = null_death_rows.iloc[:, 0:11]
X2 = pd.DataFrame(sc.fit_transform(X2), columns = X2.columns)

In [107]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2, random_state=0)

In [119]:
def testWithModel(model, X_train, X_test, y_train, y_test):
  clf = model
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)

  print(f'MAE = {mean_absolute_error(y_test, y_pred)}') 
  print(f'MAPE = {mean_absolute_percentage_error(y_test, y_pred)}')
  print(f'RMSE = {mean_squared_error(y_test, y_pred)}')

  # plot_results(y_test, y_pred)

  return y_pred

In [112]:
RFparams2={'n_estimators':[50],
          'min_samples_split':[16],
          'min_samples_leaf': [4],
           'max_features': [6],
           'max_depth': [20],
           'bootstrap': [True]}

rf = RandomForestRegressor(random_state = 21)
testWithModel(GridSearchCV(rf, RFparams2, refit=True) ,X_train, X_test, y_train, y_test )

MAE = 0.46039507158959986
MAPE = 0.13612173224098187
RMSE = 0.8987656321839417


In [69]:
testWithModel(Lasso(alpha=0.5), X_train, X_test, y_train, y_test)

MAE = 2.777563760826399
MAPE = 3.784858071420441


In [70]:
testWithModel(RidgeCV(),X_train, X_test, y_train, y_test)

MAE = 2.747641137627119
MAPE = 4.038622420365689


In [71]:
testWithModel(KNeighborsRegressor(),X_train, X_test, y_train, y_test)

MAE = 2.719313684793137
MAPE = 4.192761817399197


In [72]:
testWithModel(DecisionTreeRegressor(criterion="absolute_error"),X_train, X_test, y_train, y_test)

MAE = 4.304358655043587
MAPE = 5.092789480567514


In [104]:
testWithModel(ElasticNetCV(random_state=0), X_train, X_test, y_train, y_test)

MAE = 0.8897986325802388
MAPE = 0.22393422581680358


In [74]:
testWithModel(SVR(C=300, epsilon=0.8),X_train, X_test, y_train, y_test)

MAE = 3.685610259644844
MAPE = 4.438776918497148


Fill death col using random forest

In [ ]:
null_death_rows['DEATH [d from CT]'] = clf_random_forest.predict(X2)
df = pd.concat([non_null_death_rows,null_death_rows])
y = df.iloc[:,11]
X = df.iloc[:, 0:11]

In [ ]:
print(X.shape)

In [121]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2)

In [122]:
knn = KNeighborsRegressor(n_neighbors=9)
param_grid2 = dict(weights = ["uniform", "distance"])
knn_pred = testWithModel(GridSearchCV(knn, param_grid2),X_train, X_test, y_train, y_test)

MAE = 0.3373124321409
MAPE = 25.574692234171945
RMSE = 0.6685887576408439


In [123]:
svr_pred = testWithModel(SVR(C=300, epsilon=0.8),X_train, X_test, y_train, y_test)

MAE = 0.8219293990837235
MAPE = 22.7164988602485
RMSE = 0.9470334824737148


In [130]:
lasso_pred = testWithModel(Lasso(alpha=0.1), X_train, X_test, y_train, y_test)

MAE = 0.3777356942736409
MAPE = 25.363585992911695
RMSE = 0.6248565753524965


In [125]:
knn_pred = testWithModel(KNeighborsRegressor(),X_train, X_test, y_train, y_test)

MAE = 0.3403772681691468
MAPE = 25.97534631530856
RMSE = 0.7216121912212927


In [126]:
elastic_net_pred = testWithModel(ElasticNetCV(random_state=0), X_train, X_test, y_train, y_test)

MAE = 0.3748046375630678
MAPE = 25.580663406006014
RMSE = 0.6264942588318286


In [128]:
dtree_pred = testWithModel(DecisionTreeRegressor(criterion="absolute_error"),X_train, X_test, y_train, y_test)

MAE = 0.3761834903189418
MAPE = 25.27411700526631
RMSE = 1.378670395820654


Build Ensemble

In [161]:
array_preds = np.array([ dtree_pred, elastic_net_pred, knn_pred, lasso_pred,dtree_pred])
y_ens = np.mean((array_preds), axis=0 )

In [162]:
print(mean_absolute_error(y_ens, y_test))

0.3635115292706672


In [163]:
print(array_preds.shape)

(5, 1776)


In [169]:
import random
y_ens=[]

for i,y in enumerate(y_test):

  j = random.randint(0, 4)
  y_ens.append( array_preds[j][i])

In [172]:
len(y_ens)

1776

In [173]:
print(mean_absolute_error(y_ens, y_test))

0.37534293123365936
